In [ ]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
from scipy.spatial.distance import cdist

In [ ]:
import pandas as pd
import numpy as np
import random
from sklearn.preprocessing import StandardScaler
transform=transforms.Compose([
                           transforms.ToTensor(),transforms.Normalize((0.1307,), (0.3081,))])
sc_X = StandardScaler()


def fileFeedforGNN(name):

  data = pd.read_table(name)
  X_array = data.iloc[:, 0:16].values

  # X_array = sc_X.fit_transform(X_array)
  # df = pd.DataFrame(data = X_array)

 
  adding_process_list = []
  
  for i in range(len(data)- 27):
    X_array = data.iloc[i:i+28, 0:16].values
  
    adding_process_list.append(X_array)
  
  
  

  y = data.iloc[:,-3].values
  
  
  print(np.unique(y))


  graphList = []
  X = adding_process_list
  for i in range(len(X)):
    interList = []
    
    for j in range(len(X[i])):
      
      for k in range(len(X[i][j])):
        interList.append(X[i][j][k])
      interList+=[X[i][j][1],X[i][j][2],X[i][j][3],X[i][j][5],X[i][j][6],X[i][j][7],X[i][j][8],X[i][j][10],X[i][j][11],X[i][j][12],X[i][j][13],X[i][j][15]]
    int_array = np.array(interList)
    int_array = sc_X.fit_transform(int_array.reshape(28,28))

    #print(int_array.reshape(28,28).shape)

    

    if(y[i] =='bot'):
      graphList.append((transform(int_array),0))
    elif (y[i] == 'clothes'):
      graphList.append((transform(int_array),1))
    elif (y[i] == 'plastic'):
      graphList.append((transform(int_array),2))
    elif (y[i] == 'steel'):
      graphList.append((transform(int_array),3))
  
  return graphList


############################## data loading for 3.5 inch#####################

graphList_lablight_3_5 = fileFeedforGNN('lablight_3.5_.txt')
graphList_night_3_5 = fileFeedforGNN('night_3.5_.txt')
graphList_sunny_3_5 = fileFeedforGNN('sunny_3.5_.txt')


lablight_train_len = int(len(graphList_lablight_3_5)*0.7)
night_train_len = int(len(graphList_night_3_5)*0.7)
sunny_tarin_len = int(len(graphList_sunny_3_5)*0.7)


batch_size=64
test_batch_size=1000

random.shuffle(graphList_lablight_3_5)
random.shuffle(graphList_night_3_5)
random.shuffle(graphList_sunny_3_5)

train_loader_lablight_3_5 = torch.utils.data.DataLoader(graphList_lablight_3_5[0:lablight_train_len],batch_size)  
test_loader_lablight_3_5 = torch.utils.data.DataLoader(graphList_lablight_3_5[lablight_train_len:],test_batch_size) 

train_loader_night_3_5 = torch.utils.data.DataLoader(graphList_night_3_5[0:night_train_len],batch_size)  
test_loader_night_3_5 = torch.utils.data.DataLoader(graphList_night_3_5[night_train_len:],test_batch_size) 

train_loader_sunny_3_5 = torch.utils.data.DataLoader(graphList_sunny_3_5[0:sunny_tarin_len],batch_size)  
test_loader_sunny_3_5 = torch.utils.data.DataLoader(graphList_sunny_3_5[sunny_tarin_len:],test_batch_size) 

print(len(graphList_lablight_3_5))
print(len(graphList_night_3_5))
print(len(graphList_sunny_3_5))



############################## data loading for 7 inch#####################

graphList_lablight_7= fileFeedforGNN('lablight_7_.txt')
graphList_night_7 = fileFeedforGNN('night_7_.txt')
graphList_sunny_7 = fileFeedforGNN('sunny_7_.txt')


lablight_train_len = int(len(graphList_lablight_7)*0.7)
night_train_len = int(len(graphList_night_7)*0.7)
sunny_tarin_len = int(len(graphList_sunny_7)*0.7)


batch_size=64
test_batch_size=1000

random.shuffle(graphList_lablight_7)
random.shuffle(graphList_night_7)
random.shuffle(graphList_sunny_7)

train_loader_lablight_7 = torch.utils.data.DataLoader(graphList_lablight_7[0:lablight_train_len],batch_size)  
test_loader_lablight_7 = torch.utils.data.DataLoader(graphList_lablight_7[lablight_train_len:],test_batch_size) 

train_loader_night_7 = torch.utils.data.DataLoader(graphList_night_7[0:night_train_len],batch_size)  
test_loader_night_7 = torch.utils.data.DataLoader(graphList_night_7[night_train_len:],test_batch_size) 

train_loader_sunny_7= torch.utils.data.DataLoader(graphList_sunny_7[0:sunny_tarin_len],batch_size)  
test_loader_sunny_7= torch.utils.data.DataLoader(graphList_sunny_7[sunny_tarin_len:],test_batch_size) 

print(len(graphList_lablight_7))
print(len(graphList_night_7))
print(len(graphList_sunny_7))


['bot' 'clothes' 'plastic' 'steel']
['bot' 'clothes' 'plastic' 'steel']
['bot' 'clothes' 'plastic' 'steel']
4189
3680
3667
['bot' 'clothes' 'plastic' 'steel']
['bot' 'clothes' 'plastic' 'steel']
['bot' 'clothes' 'plastic' 'steel']
3957
3908
3550


In [ ]:
import random
class BorisNet(nn.Module):
    def __init__(self):
        super(BorisNet, self).__init__()
        self.fc = nn.Linear(784, 4, bias=False)

    def forward(self, x):
        return self.fc(x.view(x.size(0), -1))


class BorisConvNet(nn.Module):
    def __init__(self):
        super(BorisConvNet, self).__init__()
        self.conv = nn.Conv2d(1, 4, 28, stride=1, padding=14)
        self.fc = nn.Linear(4 * 4 * 4, 4, bias=False)

    def forward(self, x):
        x = F.relu(self.conv(x))
        x = F.max_pool2d(x, 7)
        return self.fc(x.view(x.size(0), -1))

class BorisGraphNet(nn.Module):
    def __init__(self, img_size=28, pred_edge=False):
        super(BorisGraphNet, self).__init__()
        self.pred_edge = pred_edge
        N = img_size ** 2
        self.fc = nn.Linear(N, 4, bias=False)
        if pred_edge:
            col, row = np.meshgrid(np.arange(img_size), np.arange(img_size))
            coord = np.stack((col, row), axis=2).reshape(-1, 2)
            coord = (coord - np.mean(coord, axis=0)) / (np.std(coord, axis=0) + 1e-5)
            coord = torch.from_numpy(coord).float()  # 784,2
            coord = torch.cat((coord.unsqueeze(0).repeat(N, 1,  1),
                                    coord.unsqueeze(1).repeat(1, N, 1)), dim=2)
            #coord = torch.abs(coord[:, :, [0, 1]] - coord[:, :, [2, 3]])
            self.pred_edge_fc = nn.Sequential(nn.Linear(4, 64),
                                              nn.ReLU(),
                                              nn.Linear(64, 1),
                                              nn.Tanh())
            self.register_buffer('coord', coord)
        else:
            # precompute adjacency matrix before training
            A = self.precompute_adjacency_images(img_size)
            self.register_buffer('A', A)


    @staticmethod
    def precompute_adjacency_images(img_size):
        col, row = np.meshgrid(np.arange(img_size), np.arange(img_size))
        coord = np.stack((col, row), axis=2).reshape(-1, 2) / img_size
        dist = cdist(coord, coord)  
        sigma = 0.05 * np.pi
        
        # Below, I forgot to square dist to make it a Gaussian (not sure how important it can be for final results)
        A = np.exp(- dist / sigma ** 2)
        print('WARNING: try squaring the dist to make it a Gaussian')
            
        A[A < 0.01] = 0
        A = torch.from_numpy(A).float()

        # Normalization as per (Kipf & Welling, ICLR 2017)
        D = A.sum(1)  # nodes degree (N,)
        D_hat = (D + 1e-5) ** (-0.5)
        A_hat = D_hat.view(-1, 1) * A * D_hat.view(1, -1)  # N,N

        # Some additional trick I found to be useful
        A_hat[A_hat > 0.0001] = A_hat[A_hat > 0.0001] - 0.2

        print(A_hat[:4, :4])
        return A_hat

    def forward(self, x):
        B = x.size(0)
        if self.pred_edge:
            self.A = self.pred_edge_fc(self.coord).squeeze()

        avg_neighbor_features = (torch.bmm(self.A.unsqueeze(0).expand(B, -1, -1),
                                 x.view(B, -1, 1)).view(B, -1))
        return self.fc(avg_neighbor_features)


def train(log_interval,model, device, train_loader, optimizer, epoch):
    model = model.double()
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
     
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval  == 0 and epoch == 150:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))


def test(model, device, test_loader,epoch):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    if(epoch == 150):

      print(
          '\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
              test_loss, correct, len(test_loader.dataset),
              100. * correct / len(test_loader.dataset)))















def mainTrainer(train_loader,test_loader):

  epochs = 150
  lr = 0.001
  pred_edge = False
  seed=1
  log_interval=200

  use_cuda = True

  torch.manual_seed(seed)

  device = torch.device("cuda" if use_cuda else "cpu")

  kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}




   
  #model = BorisGraphNet(pred_edge=pred_edge)
  model = BorisConvNet()

  model.to(device)
  print(model)
  optimizer = optim.SGD(model.parameters(), lr=lr, weight_decay=1e-1 if model == 'conv' else 1e-4)
  print('number of trainable parameters: %d' %
        np.sum([np.prod(p.size()) if p.requires_grad else 0 for p in model.parameters()]))

  for epoch in range(1, epochs + 1):
      train(log_interval,model, device, train_loader, optimizer, epoch)
      test(model, device, test_loader,epoch)
  print("I am done")

  


# print("3.5")
# print("train lablight and test lablight")
# mainTrainer(train_loader_lablight_3_5,test_loader_lablight_3_5)
# print("train lablight and test night")
# mainTrainer(train_loader_lablight_3_5,test_loader_night_3_5)
# print("train lablight and test sunny")
# mainTrainer(train_loader_lablight_3_5,test_loader_sunny_3_5)
# print("train night and test night")
# mainTrainer(train_loader_night_3_5,test_loader_night_3_5)
# print("train night and test lablight")
# mainTrainer(train_loader_night_3_5,test_loader_lablight_3_5)
# print("train night and test sunny")
# mainTrainer(train_loader_night_3_5,test_loader_sunny_3_5)
# print("train sunny and test sunny")
# mainTrainer(train_loader_sunny_3_5,test_loader_sunny_3_5)
# print("train sunny and test night")
# mainTrainer(train_loader_sunny_3_5,test_loader_night_3_5)
# print("train sunny and test lablight")
# mainTrainer(train_loader_sunny_3_5,test_loader_lablight_3_5)




# print("7")
# print("train lablight and test lablight")
# mainTrainer(train_loader_lablight_7,test_loader_lablight_7)
# print("train lablight and test night")
# mainTrainer(train_loader_lablight_7,test_loader_night_7)
# print("train lablight and test sunny")
# mainTrainer(train_loader_lablight_7,test_loader_sunny_7)
# print("train night and test night")
# mainTrainer(train_loader_night_7,test_loader_night_7)
# print("train night and test lablight")
# mainTrainer(train_loader_night_7,test_loader_lablight_7)
# print("train night and test sunny")
# mainTrainer(train_loader_night_7,test_loader_sunny_7)
# print("train sunny and test sunny")
# mainTrainer(train_loader_sunny_7,test_loader_sunny_7)
# print("train sunny and test night")
# mainTrainer(train_loader_sunny_7,test_loader_night_7)
# print("train sunny and test lablight")
# mainTrainer(train_loader_sunny_7,test_loader_lablight_7)



# print("3.5 to 7")
# print("train lablight and test lablight")
# mainTrainer(train_loader_lablight_3_5,test_loader_lablight_7)
# print("train lablight and test night")
# mainTrainer(train_loader_lablight_3_5,test_loader_night_7)
# print("train lablight and test sunny")
# mainTrainer(train_loader_lablight_3_5,test_loader_sunny_7)
# print("train night and test night")
# mainTrainer(train_loader_night_3_5,test_loader_night_7)
# print("train night and test lablight")
# mainTrainer(train_loader_night_3_5,test_loader_lablight_7)
# print("train night and test sunny")
# mainTrainer(train_loader_night_3_5,test_loader_sunny_7)
# print("train sunny and test sunny")
# mainTrainer(train_loader_sunny_3_5,test_loader_sunny_7)
# print("train sunny and test night")
# mainTrainer(train_loader_sunny_3_5,test_loader_night_7)
# print("train sunny and test lablight")
# mainTrainer(train_loader_sunny_3_5,test_loader_lablight_7)




print("7 to 3.5")
print("train lablight and test lablight")
mainTrainer(train_loader_lablight_7,test_loader_lablight_3_5)
print("train lablight and test night")
mainTrainer(train_loader_lablight_7,test_loader_night_3_5)
print("train lablight and test sunny")
mainTrainer(train_loader_lablight_7,test_loader_sunny_3_5)
print("train night and test night")
mainTrainer(train_loader_night_7,test_loader_night_3_5)
print("train night and test lablight")
mainTrainer(train_loader_night_7,test_loader_lablight_3_5)
print("train night and test sunny")
mainTrainer(train_loader_night_7,test_loader_sunny_3_5)
print("train sunny and test sunny")
mainTrainer(train_loader_sunny_7,test_loader_sunny_3_5)
print("train sunny and test night")
mainTrainer(train_loader_sunny_7,test_loader_night_3_5)
print("train sunny and test lablight")
mainTrainer(train_loader_sunny_7,test_loader_lablight_3_5)


7 to 3.5
train lablight and test lablight
BorisConvNet(
  (conv): Conv2d(1, 4, kernel_size=(28, 28), stride=(1, 1), padding=(14, 14))
  (fc): Linear(in_features=64, out_features=4, bias=False)
)
number of trainable parameters: 3396
Train Epoch: 150 [0/2769 (0%)]	Loss: 0.056663

Test set: Average loss: 2.5299, Accuracy: 332/1257 (26%)

I am done
train lablight and test night
BorisConvNet(
  (conv): Conv2d(1, 4, kernel_size=(28, 28), stride=(1, 1), padding=(14, 14))
  (fc): Linear(in_features=64, out_features=4, bias=False)
)
number of trainable parameters: 3396
Train Epoch: 150 [0/2769 (0%)]	Loss: 0.056663

Test set: Average loss: 2.3448, Accuracy: 302/1104 (27%)

I am done
train lablight and test sunny
BorisConvNet(
  (conv): Conv2d(1, 4, kernel_size=(28, 28), stride=(1, 1), padding=(14, 14))
  (fc): Linear(in_features=64, out_features=4, bias=False)
)
number of trainable parameters: 3396
Train Epoch: 150 [0/2769 (0%)]	Loss: 0.056663

Test set: Average loss: 2.5550, Accuracy: 243/1101 